# Model Reviewer
Allows for the passing of a model generator function to a reviewer object that will generate the model, render it, display, and allow the option to reject or save the model.

In [1]:
import structgeo.model as geo
import structgeo.plot as geovis
import structgeo.probability as rv

import numpy as np
import pyvista as pv

pv.set_jupyter_backend('static')

## Model
Specify the model type first, then populate it with a history

In [2]:
# Set random seed for reproducibility
# random_seed = 1234
# np.random.seed(random_seed)

In [3]:
# Start with some model specifications
resolution = 128
# Cubic model keeps 
max_x = 10
min_x = - max_x
model_size = max_x - min_x
bounds = (min_x,max_x)
model = geo.GeoModel(bounds, resolution)
print(f"Model created with bounds {bounds} and resolution {resolution}")

Model created with bounds (-10, 10) and resolution 128


##

In [4]:
from structgeo.plot import ModelGenerator
from structgeo.data import FileManager

DEFAULT_BASE_DIR = "../saved_models"

# Here we define the generation of a history and apply it to the model
def generate_model():
    # Bedrock layer
    base = min_x + model_size / 4 + np.random.normal(0, model_size / 10)
    value = 1
    bedrock = geo.Bedrock(base,value)

    # Random sedimentary layer filled to around 1/2 of the model
    total_thickness =  model_size / 2 + np.random.normal(0, model_size / 10)

    # Generate random values and thicknesse
    sb = geo.SedimentBuilder(start_value= 2, total_thickness=total_thickness, min_layers= 3, max_layers= 10, std= 0.5)
        
    sediment = geo.Sedimentation(*sb.get_layers())
    
    model.clear_history()
    model.add_history([bedrock, sediment])
    model.compute_model()
    return model

reviewer = ModelGenerator(generate_model_func=generate_model, base_dir=DEFAULT_BASE_DIR)
reviewer.start_review()

Output()

Model height adjusted.


Model height adjusted.


Model height adjusted.


Model height adjusted.


Model height adjusted.
Model discarded.


Model height adjusted.
Model discarded.


Model height adjusted.
Model discarded.


Model height adjusted.


Model height adjusted.
Model discarded.
